In [1]:
import time
import warnings
import logging
import tensorflow as tf

In [2]:
@tf.function
def add(a, b):
    return a + b

@tf.function
def sub(a, b):
    return a - b

@tf.function
def mul(a, b):
    return a * b

@tf.function
def dev(a, b):
    return a / b

In [3]:
print(add(tf.constant(5), tf.constant(2)))

tf.Tensor(7, shape=(), dtype=int32)


2022-08-29 12:55:29.242267: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
@tf.function
def matmul(a, b):
    return tf.matmul(a, b)

In [12]:
@tf.function
def linear(m, x, c):
    return add(matmul(m, x), c)

In [7]:
m = tf.constant([[4.0, 5.0, 6.0]], tf.float32)
m

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[4., 5., 6.]], dtype=float32)>

In [9]:
x = tf.Variable([[100.0], [100.0], [100.0]], tf.float32)
x

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[100.],
       [100.],
       [100.]], dtype=float32)>

In [10]:
c = tf.constant([[1.0]], tf.float32)
c

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1.]], dtype=float32)>

In [13]:
linear(m, x, c)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1501.]], dtype=float32)>

In [14]:
@tf.function
def pos_neg_check(x):
    reduce_sum = tf.reduce_sum(x)
    
    if reduce_sum > 0:
        return tf.constant(1)
    
    elif reduce_sum == 0:
        return tf.constant(0)
    
    else: 
        return tf.constant(-1)

In [16]:
pos_neg_check(tf.constant([100, 100]))

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [17]:
pos_neg_check(tf.constant([100, -100]))

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [18]:
pos_neg_check(tf.constant([100, -100, -100]))

<tf.Tensor: shape=(), dtype=int32, numpy=-1>

In [19]:
num = tf.Variable(7)

In [22]:
@tf.function
def add_times(x):
    for i in tf.range(x):
        num.assign_add(x)

In [23]:
add_times(5)

In [24]:
print(num)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=32>


In [25]:
a = tf.Variable(1.0)
b = tf.Variable(2.0)

In [26]:
@tf.function
def f(x, y):
    a.assign(y * b)
    b.assign_add(x * a)
    return a + b

In [27]:
f(1, 2)

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

In [28]:
@tf.function
def square(a):
    print("Input a: ", a)
    return a * a

In [31]:
x = tf.Variable([[2, 2], [2, 2]], dtype = tf.float32)
square(x)

Input a:  <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32>


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[4., 4.],
       [4., 4.]], dtype=float32)>

In [32]:
y = tf.Variable([[2,3], [4,5]], dtype= tf.int32)
square(y)

Input a:  <tf.Variable 'Variable:0' shape=(2, 2) dtype=int32>


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 4,  9],
       [16, 25]], dtype=int32)>

In [34]:
z = tf.Variable([[1,2], [3,4]], dtype = tf.int32)
square(z)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 1,  4],
       [ 9, 16]], dtype=int32)>

In [37]:
concrete_int_square_fn = square.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.int32))
concrete_int_square_fn

Input a:  Tensor("a:0", dtype=int32)


<ConcreteFunction square(a) at 0x15C0E6240>

In [38]:
concrete_int_square_fn(tf.constant([[2,2]], dtype=tf.int32))

<tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[4, 4]], dtype=int32)>

In [39]:
@tf.function
def f(x):
    print("Python execution: ", x)
    tf.print("Graph execution: ", x)

In [40]:
f(1)

Python execution:  1
Graph execution:  1


In [41]:
f(2)

Python execution:  2
Graph execution:  2


In [42]:
f(1)

Graph execution:  1


In [43]:
f(3)

Python execution:  3
Graph execution:  3


In [44]:
f("hello")

Python execution:  hello
Graph execution:  hello


In [45]:
f("string")

Python execution:  string
Graph execution:  string


In [46]:
f(3)

Graph execution:  3


In [47]:
f(2)

Graph execution:  2


In [51]:
def fn_with_variable_init_eager():
    a = tf.constant([[10, 10], [11.,1.]])
    x = tf.constant([[1.,0.] ,[0.,1.]])
    b = tf.Variable(12.)
    
    y = tf.matmul(a, x) + b
    
    tf.print("tf_print: ", y)
    return y

In [52]:
fn_with_variable_init_eager()

tf_print:  [[22 22]
 [23 13]]


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[22., 22.],
       [23., 13.]], dtype=float32)>

In [53]:
@tf.function
def fn_with_variable_init_autograph():
    a = tf.constant([[10, 10], [11.,1.]])
    x = tf.constant([[1.,0.] ,[0.,1.]])
    b = tf.Variable(12.)
    
    y = tf.matmul(a, x) + b
    
    tf.print("tf_print: ", y)
    return y

In [54]:
fn_with_variable_init_autograph()

ValueError: in user code:

    File "/var/folders/c1/2ltzp02n15z_6jjqlpm151gc0000gn/T/ipykernel_86033/3204247165.py", line 5, in fn_with_variable_init_autograph  *
        b = tf.Variable(12.)

    ValueError: tf.function only supports singleton tf.Variables created on the first call. Make sure the tf.Variable is only created once or created outside tf.function. See https://www.tensorflow.org/guide/function#creating_tfvariables for more information.


In [55]:
class F():
    def __init__(self):
        self._b = None
    
    @tf.function
    def __call__(self):
        a = tf.constant([[10, 10], [11., 1.]])
        x = tf.constant([[1., 0.], [0., 1.]])
        
        if self._b is None:
            self._b = tf.Variable(10.)
        
        y = tf.matmul(a, x) + self._b
        print(y)
        
        tf.print("tf_print: ", y)
        return y

fn_with_variable_init_autograph = F()
fn_with_variable_init_autograph()

Tensor("add:0", shape=(2, 2), dtype=float32)
Tensor("add:0", shape=(2, 2), dtype=float32)
tf_print:  [[20 20]
 [21 11]]


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[20., 20.],
       [21., 11.]], dtype=float32)>

In [56]:
def f(x): 
    if x > 0:
        x *= x
    return x
print(tf.autograph.to_code(f))

def tf__f(x):
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def if_body():
            nonlocal x
            x = ag__.ld(x)
            x *= x

        def else_body():
            nonlocal x
            pass
        ag__.if_stmt((ag__.ld(x) > 0), if_body, else_body, get_state, set_state, ('x',), 1)
        try:
            do_return = True
            retval_ = ag__.ld(x)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



In [62]:
@tf.function
def g(x):
    return x

start = time.time()
for i in tf.range(2000):
    g(i)
end = time.time()
print("tf.Tensor time elapsed: ", (end-start))

tf.Tensor time elapsed:  1.2978630065917969


In [58]:
warnings.filterwarnings('ignore')
logging.getLogger('tensorflow').disabled = True

In [63]:
start = time.time()

print("starting")

for i in range(2000):
    g(i)

end = time.time()

print("Native type time elapsed: ", (end-start))

starting
Native type time elapsed:  37.31594705581665
